# Notebook para análisis de precios promedio

Ideas: 
- Poder realizar diferenciación de precios.
- Encontrar una aproximación de precios por categoría para asociarlo con el nivel de adversión al riesgo en cada tienda del ANTAD.

In [20]:
# Importar librerías 
    # Manejo de información en arreglos y dataframes
import numpy as np
import pandas as pd
    # Manejo de información geoespacial
import geopandas as gpd
import pyproj
from shapely.ops import transform
    # Manejo de expresiones regulares
import re
    # Manejo de unicode
from unidecode import unidecode
    # Manejo de archivos y directorios del Sistema Operativo
import os
    # Visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

# Definimos directorios automatizados
CURRENT_PATH = os.getcwd()
GENERAL_PATH = os.path.dirname(CURRENT_PATH)
DATA_PATH = os.path.join(GENERAL_PATH, "data")
OUTPUT_PATH = os.path.join(GENERAL_PATH, "output")

In [10]:
# Leemos los datos

data = pd.read_csv(os.path.join(DATA_PATH, "INP_PP.csv"), encoding="latin-1", skiprows=5)
data.drop(columns={"Año", "Mes", "Estatus", "Fecha_Pub_DOF"}, inplace=True)

In [12]:
data.head(3)

,Clave ciudad,Nombre ciudad,División,Grupo,Clase,Subclase,Clave genérico,Genérico,Consecutivo,Especificación,Precio promedio,Cantidad,Unidad
0,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",04 Arroz y cereales preparados,1,Arroz,1,"VERDE VALLE, BLANCO, SUPER EXTRA, BOLSA DE 900...",44.44,1,KG
1,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",04 Arroz y cereales preparados,1,Arroz,2,"VERDE VALLE, BLANCO, SUPER EXTRA, BOLSA DE 1 K...",38.00,1,KG
2,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",04 Arroz y cereales preparados,1,Arroz,3,"SOS, BLANCO, EXTRA, BOLSA DE 1 KG ...",32.00,1,KG


In [15]:
data.División.unique()

array(['1. Alimentos, bebidas y tabaco', '2. Ropa, calzado y accesorios',
       '3. Vivienda', '4. Muebles, aparatos y accesorios domésticos',
       '5. Salud y cuidado personal', '6. Transporte',
       '8. Otros servicios', '7. Educación y esparcimiento'], dtype=object)

In [16]:
data.Grupo.unique()

array(['1.1. Alimentos', '1.2. Bebidas alcohólicas y tabaco', '2.1. Ropa',
       '2.3. Accesorios y cuidados del vestido', '2.2. Calzado',
       '3.1. Costo de uso de vivienda',
       '3.3. Otros servicios relacionados con la vivienda',
       '3.2. Electricidad y combustibles',
       '4.1. Muebles y aparatos domésticos',
       '4.2. Accesorios y artículos de limpieza para el hogar',
       '5.2. Cuidado personal', '5.1. Salud',
       '6.2. Transporte por cuenta propia', '6.1. Transporte público',
       '8.1. Otros servicios', '7.2. Esparcimiento', '7.1. Educación'],
      dtype=object)

In [25]:
precio_ciudad = data.groupby(["Clave ciudad", "Nombre ciudad", "División", "Grupo"])[['Precio promedio']].mean()
precio_ciudad.to_csv(os.path.join(OUTPUT_PATH, "precios_por_ciudad.csv"), encoding='utf-8-sig')
precio_ciudad

Precio promedio
Clave ciudad Nombre ciudad                 División                       Grupo                                                  
1            Área Met. de la Cd. de México 1. Alimentos, bebidas y tabaco 1.1. Alimentos                               116.977566
                                                                          1.2. Bebidas alcohólicas y tabaco            399.011121
                                           2. Ropa, calzado y accesorios  2.1. Ropa                                    576.702149
                                                                          2.2. Calzado                                 686.860094
                                                                          2.3. Accesorios y cuidados del vestido      1498.008235
...                                                                                                                           ...
55           Zacatecas, Zac.               6. Transporte                  6.1. Transporte público                      834.960000
                                                                          6.2. Transporte por cuenta propia          56196.483459
                                           7. Educación y esparcimiento   7.1. Educación                              2339.251765
                                                                          7.2. Esparcimiento                          1796.645033
                                           8. Otros servicios             8.1. Otros servicios                        1951.052889

[935 rows x 1 columns]

In [26]:
precios_total = data.groupby(["División", "Grupo"])[['Precio promedio']].mean()
precios_total.to_csv(os.path.join(OUTPUT_PATH, "precios_total.csv"), encoding='utf-8-sig')
precios_total

Precio promedio
División                                     Grupo                                                              
1. Alimentos, bebidas y tabaco               1.1. Alimentos                                           115.972189
                                             1.2. Bebidas alcohólicas y tabaco                        303.349897
2. Ropa, calzado y accesorios                2.1. Ropa                                                401.383510
                                             2.2. Calzado                                             614.245084
                                             2.3. Accesorios y cuidados del vestido                  1015.590401
3. Vivienda                                  3.1. Costo de uso de vivienda                            514.086998
                                             3.2. Electricidad y combustibles                        9586.822104
                                             3.3. Otros servicios relacionados con la vivienda        746.706704
4. Muebles, aparatos y accesorios domésticos 4.1. Muebles y aparatos domésticos                      6116.039444
                                             4.2. Accesorios y artículos de limpieza para el...       381.617576
5. Salud y cuidado personal                  5.1. Salud                                              1868.978376
                                             5.2. Cuidado personal                                    200.148467
6. Transporte                                6.1. Transporte público                                 1061.489992
                                             6.2. Transporte por cuenta propia                      85766.419421
7. Educación y esparcimiento                 7.1. Educación                                          2749.465136
                                             7.2. Esparcimiento                                      1877.660849
8. Otros servicios                           8.1. Otros servicios                                    2128.728034